In [77]:
import requests
from datetime import datetime
from datetime import date

In [78]:
# The teamguids for which we'll find all the oponents guids
teamguid1 = 'BVBL1117HSE++1'
teamguid2 = 'BVBL1117HSE++2'

In [79]:
def opponents_teamguids(teamguid):
    """
    Start with a teamguid and return a dictionary of all opponents teamnames + teamguids
    """
    url_teamdetail = 'http://vblcb.wisseq.eu/VBLCB_WebService/data/TeamDetailByGuid?teamguid=' + teamguid
    opponents_teamguids = {}
    
    try:
        response = requests.get(url_teamdetail)
        for teamdetail in response.json()[0]['poules'][0]['teams']:
            opponents_teamguids[teamdetail['naam']] = teamdetail['guid'].replace(" ","+") # Teamname = key, teamguid = value. Replace spaces with '+' in key 
        
        opponents_teamguids = {key:val for key, val in opponents_teamguids.items() if val != teamguid} # Copy the dictionary but remove our own teamguid
        return(opponents_teamguids)
    
    except:
        print('Invalid Response or Error: ' + str(response))
  

In [80]:
def opponent_matchguids(opponents_teamguid):
    """
    Start with a single opponents teamguid & return a list of all the matchguids that are before the current date
    """   
    url_teammatches = 'http://vblcb.wisseq.eu/VBLCB_WebService/data/TeamMatchesByGuid?teamguid=' + opponents_teamguid
    opponents_matchguids = []
    today = date.today()
    
    try:
        response = requests.get(url_teammatches)
        for teammatches in response.json():
            if datetime.strptime(teammatches['datumString'], "%d-%m-%Y").date() < today: # If matchday is before today, then append the matchguid to list
                opponents_matchguids.append(teammatches['guid'])                
        return(opponents_matchguids)
    except:
        print('Invalid Response or Error: ' + str(response))
    

In [81]:
def opponents_matches(opponents_teams):
    """
    Use the opponent_matchguids function to return a dictionary of all opponent names & all their matchguids before today
    """
    opponents_matches = {}

    for key, value in opponents_teams.items():
        opponents_matches[key] = opponent_matchguids(value)
    return(opponents_matches)

In [84]:
# Get dicts of all the opponents names + their teamguids per team
team1_opponents = opponents_teamguids(teamguid1)
team2_opponents = opponents_teamguids(teamguid2) 

# Get dicts of all the opponents names + their all their matchguids before today
team1_opponents_matches = opponents_matches(team1_opponents)
team2_opponents_matches = opponents_matches(team2_opponents)